This code is for analyzing neural MEA data and generating an objective index - neural activity score - to evaluate network ontogeny and effects of treatments/conditions/perturbation.

See Passaro et al. and the README file in the GitHub repository for more information, usage instructions, and sample data.

In [ ]:
import pandas as pd
import tkinter as tk
from tkinter import filedialog
import os
import shutil
import openpyxl as xl
import numpy as np
from scipy import stats
import sys
from collections import OrderedDict

In [ ]:
%%capture
from tqdm import tqdm
tqdm().pandas()

In [ ]:
# Ask which directory to read from, note the following file structure below:
# Experiment folder > Plate subfolder(s) > DIV subfolder(s) > Maestro.raw, StatCompiler.csv, etc...
# Select the EXPERIMENT folder
print('Select experiment folder...')
root = tk.Tk()
root.withdraw()
experimentPath = filedialog.askdirectory()
print("Experiment folder: ",experimentPath)

In [ ]:
### Compile .csv StatCompiler files into one .xlsx file ###
def compileAndMerge(experimentPath):
    # List plate subfolders
    plateFolders = os.listdir(experimentPath)

    # Loop through each plate folder
    for plateFolder in tqdm(plateFolders):
        if 'platemap' in plateFolder:	# Ignore platemap file(s)
            continue
        else:
            platePath = experimentPath + '\\' + plateFolder

        # Create Compiled plate .xlsx file
        fileout = platePath + '\\' + '1_AllDaysCompiled_' + plateFolder + '.xlsx'
        writer = pd.ExcelWriter(fileout, engine='xlsxwriter')

        # Create list of day (DIV) folders within plate folder
        divFolders = os.listdir(platePath)

        # Go into each DIV folder and list filenames
        for div in divFolders:
            files = os.listdir(platePath + '\\' + div)

        # Find .csv files
            for filename in files:
                if '.csv' in filename and 'Statistics' in filename:
                    # Copy file to destination folder
                    sourceFile = platePath + '\\' + div + '\\' + filename
                    destinationFile = platePath + '\\' + div + '_' + filename
                    shutil.copy(sourceFile, destinationFile)

                    # Merge file into Compiled .xlsx file
                    df = pd.read_csv(sourceFile, names = list(range(0,770)), header=None, engine='python')
                    df.to_excel(writer, sheet_name=div, header=False, index=False)
        writer.save()
        print('Compiled .xlsx file saved for ' + plateFolder + '.')

In [ ]:
compileAndMerge(experimentPath)

In [ ]:
### Analyze compiled .xslx files for all parameters ###
def analyzeStatCompilerFiles(experimentPath):
    # List plate subfolders
    plateFolders = os.listdir(experimentPath)
    plateFolders = [x for x in plateFolders if '.csv' not in x]
    plateFolders = [x for x in plateFolders if '.raw' not in x]
    plateFolders = [x for x in plateFolders if '.txt' not in x]
    plateFolders = [x for x in plateFolders if '.xlsx' not in x]
    plateFolders = [x for x in plateFolders if '.docx' not in x]
    plateFolders = [x for x in plateFolders if 'Compiled' not in x]
    
    # Loop through each plate folder
    for plateFolder in plateFolders:
        print('Analyzing ' + plateFolder + '...')
        if 'platemap' in plateFolder:	# Ignore platemap file(s)
            continue
        else:
            platePath = experimentPath + '\\' + plateFolder

        filename = platePath + '\\' + '1_AllDaysCompiled_' + plateFolder + '.xlsx'
        wb = xl.load_workbook(filename) 	# Load first workbook (one plate compiled, each sheet = 1 day)
        sheets = wb.sheetnames

        ##### Determine array dimensions (reps, groups, parameters, days) #####
        # Define group names
        sheet = wb.worksheets[0]    # Use first sheet as template to read from
        groups = []                 # Define empty list to store group names
        for row in range(1, sheet.max_row+1): # Loop through rows to find row containing Group/Treatment names
            if sheet.cell(row = row, column = 1).value == 'Treatment':
                groupRow = row
                break
        for column in range(2,sheet.max_column+1):
            if sheet.cell(row = groupRow, column = column).value != None and sheet.cell(row = groupRow, column = column).value not in groups:
                groups.append(str(sheet.cell(row = groupRow, column = column).value))
        
        # Determine number of groups (if not defined in first sheet, try to copy from last sheet first)
        numgroups = len(groups)
        if numgroups == 0:
            lastSheet = wb.worksheets[-1]
            for column in range(2,lastSheet.max_column+1):
                if lastSheet.cell(row = groupRow, column = column).value != None:
                    sheet.cell(row = groupRow, column = column).value = lastSheet.cell(row = groupRow, column = column).value
        for column in range(2,sheet.max_column+1):
            if sheet.cell(row = groupRow, column = column).value != None and sheet.cell(row = groupRow, column = column).value not in groups:
                groups.append(str(sheet.cell(row = groupRow, column = column).value))
        
        numgroups = len(groups)
        if numgroups == 0:
            raise Warning("Define groups in first sheet of .xlsx file ('Treatment' row) before proceeding.")
            sys.exit()
        
        # Find number of reps (assumes equal number per group)
        for row in range(1, sheet.max_row+1):
            if sheet.cell(row = row, column = 1).value == 'Total Wells':
                numWells = 0
                if sheet.cell(row = row, column = 3).value != None:
                    for i in range(2, numgroups+2):
                        numWells += int(sheet.cell(row = row, column = i).value)
                    break
                else:
                    numWells = int(sheet.cell(row = row, column = 2).value)

        numreps = 0
        for group in range(0, len(groups)):
            numrepsCounter = 0
            for column in range(2,numWells+2):
                if str(sheet.cell(row = groupRow, column = column).value) == str(groups[group]):
                    numrepsCounter += 1
            if numrepsCounter > numreps:
                numreps = numrepsCounter

        # Count number of calculated parameters
        for row in range(1, sheet.max_row+1): # Loop through rows to find row containing first parameter
            if sheet.cell(row = row, column = 1).value == 'Number of Spikes':
                firstParamRow = row
                break
        for row in range(1, sheet.max_row+1): # Loop through rows to find row containing last parameter
            if sheet.cell(row = row, column = 1).value == 'Synchrony Index':
                lastParamRow = row
                break        
                
        numparams = lastParamRow - firstParamRow + 1

        # Determine number of days
        numdays = len(sheets)

        # Create NumPy array with dimensions: reps x groups x parameters x days
        array = np.ndarray(shape=(numreps,numgroups,numparams,numdays))

        ### Array indices ###
        # Reps: 0-x in order of Excel file
        # Groups: 0-x in order of Excel file
        # Parameters: 0-x = Parameters in order of Excel file from statistics compiler
        # Days: 0-x = Days in order of compiled Excel file sheets
        #####################

        for row in range(1, sheet.max_row+1): # Loop through rows to find row containing Well names
            if sheet.cell(row = row, column = 1).value == 'Well':
                wellNameRow = row
                break
        
        columns = OrderedDict()    # Create ORDERED dictionary for columns
        wells = OrderedDict()      # Create ORDERED dictionary for wells
        for i in groups:
            columns[i] = []   # Create empty column list for each group
            wells[i] = []     # Create empty well list for each group
            for column in range(2,numWells+2):
                if sheet.cell(row = groupRow, column = column).value == i:    # Fill in column numbers and well names/numbers ('A1, A2', etc.) for each group
                    columns[i].append(column)
                    wells[i].append(str(sheet.cell(row = wellNameRow, column = column).value))
        
        ### Fill in array with all values ###
        for day in range(0, numdays):    # Loop through sheets (days)
            sheet = wb.worksheets[day]
            for row in range(1, sheet.max_row+1): # Loop through rows to find row containing first parameter
                if sheet.cell(row = row, column = 1).value == 'Number of Spikes':
                    firstParamRow = row
                    break
            for row in range(1, sheet.max_row+1): # Loop through rows to find row containing last parameter
                if sheet.cell(row = row, column = 1).value == 'Synchrony Index':
                    lastParamRow = row
                    break
            for param in range(0, numparams):   # Loop through rows (parameters)
                row = (param + firstParamRow)
                g = 0
                for group in columns:
                    rep = 0
                    for c in columns[group]:
                        if sheet.cell(row = row, column = c).value == None:
                            array[rep][g][param][day] = 0
                        else:
                            array[rep][g][param][day] = sheet.cell(row = row, column = c).value
                        rep += 1
                    g += 1
        
        # Create empty Pandas dataframe (for later conversion from NumPy array) 
        data = pd.DataFrame()
        df_write = pd.ExcelWriter(platePath + '\\' + '2_AnalyzedAllParameters_' + plateFolder + '.xlsx')
        for p in tqdm(range(0,numparams)):
            paramname = str(sheet.cell(row = (p + firstParamRow), column = 1).value)
            xdays = []
            sheets_strings = []
            for s in sheets:
                sheets_strings.append(str(s))
            for x in range(0, numdays):
                xdays.append(sheets_strings[x][3:6])

            ### Slice array for desired parameter and days then convert to pandas dataframe ###
            array_sliced = array[:,:,p,:]
            array_meshgrid = np.column_stack(list(map(np.ravel, np.meshgrid(*map(np.arange, array_sliced.shape), indexing="ij"))) + [array_sliced.ravel()])

            datanew = pd.DataFrame(array_meshgrid, columns = ['Rep', 'Group', 'Day', paramname])   # Convert numpy array to pandas dataframe
            
            datanew['Rep'] += 1

            groupnames = {}
            for i in range(0, numgroups):
                groupnames[i] = groups[i]
            datanew['Group'].replace(groupnames, inplace = True)

            daysdict = {}
            for i in range(0, numdays):
                daysdict[i] = xdays[i]
            datanew['Day'].replace(daysdict, inplace = True)

            if p == 0:
                data['Rep'] = datanew['Rep']
                data['Group'] = datanew['Group']
                data['Day'] = datanew['Day']
            data['Plate'] = plateFolder
            data[paramname] = datanew[paramname]
            data.to_excel(df_write, "All Parameters", index=False)
            #print('Analyzed Parameter ',p,'/',numparams)
        df_write.save()
        print('Analysis Excel file exported for ' + plateFolder + '.')

In [ ]:
analyzeStatCompilerFiles(experimentPath)

In [ ]:
def mergePlateOutputFiles(experimentPath):
    plateFolders = os.listdir(experimentPath)
    plateFolders = [x for x in plateFolders if '.csv' not in x]
    plateFolders = [x for x in plateFolders if '.raw' not in x]
    plateFolders = [x for x in plateFolders if '.txt' not in x]
    plateFolders = [x for x in plateFolders if '.xlsx' not in x]
    plateFolders = [x for x in plateFolders if '.docx' not in x]
    plateFolders = [x for x in plateFolders if 'Compiled' not in x]
    
    compiledPath = experimentPath + "\\Compiled"
    if not os.path.exists(compiledPath):
        os.makedirs(compiledPath)
    
    # Check if plates are identical
    identicalPlates = input("Are plates identical [y/n]? ")
    if identicalPlates == 'y':
        numplates = int(input('How many identical plates are there [int]? '))
        if not isinstance(numplates, int):
            raise Warning("Invalid response. Answer must be an integer. Exiting...")
            sys.exit()
    elif identicalPlates == 'n':
        pass
    else:
        raise Warning("Invalid response. Answer must be 'y' or 'n'. Exiting...")
        sys.exit()
    
    files2_AnalyzedAllParameters = []

    for plateFolder in plateFolders:
        if 'platemap' in plateFolder:	# Ignore platemap file(s)
            continue
        else:
            platePath = experimentPath + '\\' + plateFolder
        # Create lists of .xlsx filenames to be merged
        files2_AnalyzedAllParameters.append(platePath + '\\' + '2_AnalyzedAllParameters_' + plateFolder + '.xlsx')
    
    # Open .xlsx file(s) and fix rep numbers for concatenation (e.g. Reps 1-6 on 2 plates become Reps 1-12 in compiled file)
    if identicalPlates == 'y':
        for i in range(1,len(files2_AnalyzedAllParameters)):
            wb = xl.load_workbook(files2_AnalyzedAllParameters[i])
            sheet = wb.worksheets[0]
            repCol = 0
            for column in range(1,sheet.max_column+1):
                if sheet.cell(row = 1, column = column).value == 'Rep':
                    repCol = column
            maxRep = 0
            for row in range(2, sheet.max_row+1):
                if sheet.cell(row = row, column = repCol).value > maxRep:
                    maxRep = sheet.cell(row = row, column = repCol).value
            for row in range(2, sheet.max_row+1):
                sheet.cell(row = row, column = repCol).value += (maxRep * i)
            wb.save(compiledPath + '\\2_AnalyzedAllParameters_' + plateFolders[i] + '_RepsCorrected.xlsx')
            files2_AnalyzedAllParameters[i] = compiledPath + '\\2_AnalyzedAllParameters_' + plateFolders[i] + '_RepsCorrected.xlsx'
    
    # Read .xlsx files as pandas dataframes, then append to lists for concatenation
    list2_AnalyzedAllParameters = []

    for file in files2_AnalyzedAllParameters:
        list2_AnalyzedAllParameters.append(pd.read_excel(file))
    
    # Concatenate dataframes
    df2 = pd.concat(list2_AnalyzedAllParameters)

    # Export to .xlsx
    df2.to_excel(compiledPath + "\\2_AnalyzedAllParameters_Compiled.xlsx", index=False)
    print("Excel files merged.")

In [ ]:
mergePlateOutputFiles(experimentPath)

In [ ]:
def normZscore(experimentPath):
    # List plate subfolders
    plateFolders = os.listdir(experimentPath)
    plateFolders = [x for x in plateFolders if '.csv' not in x]
    plateFolders = [x for x in plateFolders if '.raw' not in x]
    plateFolders = [x for x in plateFolders if '.txt' not in x]
    plateFolders = [x for x in plateFolders if '.xlsx' not in x]
    plateFolders = [x for x in plateFolders if '.docx' not in x]
    
    # Loop through each plate folder
    for plateFolder in plateFolders:
        if 'platemap' in plateFolder:	# Ignore platemap file(s)
            continue
        else:
            platePath = experimentPath + '\\' + plateFolder	

        wb = xl.load_workbook(platePath + '\\' + '2_AnalyzedAllParameters_' + plateFolder + '.xlsx') 		# Load workbook
        sheets = wb.sheetnames							# Create list of all worksheets

        for page in range(0, len(sheets)):	 			# Loop through sheets (days)
            sheet = wb.worksheets[page]
            for col in range(5,sheet.max_column+1):
                colvals = []
                for row in range(2,sheet.max_row+1):
                    if sheet.cell(row = row, column = col).value == None:
                        pass
                    elif type(sheet.cell(row = row, column = col).value) == str:
                        pass	
                    else:
                        colvals.append(sheet.cell(row = row, column = col).value)
                col_mean = np.mean(colvals)
                col_std = np.std(colvals)
                for row in range(2,sheet.max_row+1):
                    if sheet.cell(row = row, column = col).value == None:
                        pass
                    elif type(sheet.cell(row = row, column = col).value) == str:
                        pass
                    else:
                        sheet.cell(row = row, column = col).value = ((sheet.cell(row = row, column = col).value - col_mean) / col_std)

        wb.save(platePath + '\\' + '3_Zscore_' + plateFolder + '.xlsx')
        print('Z-score Excel file exported for ' + plateFolder + '.')

In [ ]:
normZscore(experimentPath)

In [ ]:
def NAS(experimentPath):
    # List plate subfolders
    plateFolders = os.listdir(experimentPath)
    plateFolders = [x for x in plateFolders if '.csv' not in x]
    plateFolders = [x for x in plateFolders if '.raw' not in x]
    plateFolders = [x for x in plateFolders if '.txt' not in x]
    plateFolders = [x for x in plateFolders if '.xlsx' not in x]
    plateFolders = [x for x in plateFolders if '.docx' not in x]
    
    # Loop through each plate folder
    for plateFolder in plateFolders:
        if 'platemap' in plateFolder:	# Ignore platemap file(s)
            continue
        else:
            platePath = experimentPath + '\\' + plateFolder

        wb = xl.load_workbook(platePath + '\\' + '3_Zscore_' + plateFolder + '.xlsx') 		# Load workbook
        sheets = wb.sheetnames   # Create list of all worksheets
        sheet = wb.worksheets[0]

        for col in range(1,sheet.max_column+1):
            if sheet.cell(row = 1, column = col).value == 'Mean Firing Rate (Hz)':
                meanFiringRate = col
                continue
            elif sheet.cell(row = 1, column = col).value == 'ISI Coefficient of Variation':
                ISIcoeffVar = col
                continue
            elif sheet.cell(row = 1, column = col).value == 'ISI Coefficient of Variation - Avg':
                ISIcoeffVar = col
                continue
            elif sheet.cell(row = 1, column = col).value == 'Number of Bursting Electrodes':
                numBurstingElecs = col
                continue
            elif sheet.cell(row = 1, column = col).value == 'Burst Duration - Avg (s)':
                burstDuration = col
                continue                
            elif sheet.cell(row = 1, column = col).value == 'Number of Spikes per Burst - Avg':
                numSpikesPerBurst = col
                continue
            elif sheet.cell(row = 1, column = col).value == 'Mean ISI within Burst - Avg':
                meanISIwithinBurst = col
                continue                
            elif sheet.cell(row = 1, column = col).value == 'Median ISI within Burst - Avg':
                medianISIwithinBurst = col
                continue                
            elif sheet.cell(row = 1, column = col).value == 'Inter-Burst Interval - Avg (s)':
                IBI = col
                continue                
            elif sheet.cell(row = 1, column = col).value == 'Burst Frequency - Avg (Hz)':
                burstFreq = col
                continue                
            elif sheet.cell(row = 1, column = col).value == 'Normalized Duration IQR - Avg':
                normDurationIQR = col
                continue                
            elif sheet.cell(row = 1, column = col).value == 'IBI Coefficient of Variation - Avg':
                IBIcoeffVar = col
                continue                
            elif sheet.cell(row = 1, column = col).value == 'Burst Percentage - Avg':
                burstPercentage = col
                continue                
            elif sheet.cell(row = 1, column = col).value == 'Network Burst Frequency (Hz)':
                networkBurstFreq = col
                continue                
            elif sheet.cell(row = 1, column = col).value == 'Network Burst Duration - Avg (sec)':
                networkBurstDuration = col
                continue
            elif sheet.cell(row = 1, column = col).value == 'Number of Spikes per Network Burst - Avg':
                numSpikesPerNetworkBurst = col
                continue                
            elif sheet.cell(row = 1, column = col).value == 'Number of Elecs Participating in Burst - Avg':
                numElecsParticipatingInBurst = col
                continue                
            elif sheet.cell(row = 1, column = col).value == 'Number of Spikes per Network Burst per Channel - Avg':
                numSpikesPerNetworkBurstPerChannel = col
                continue                
            elif sheet.cell(row = 1, column = col).value == 'Network Burst Percentage':
                networkBurstPercentage = col
                continue                
            elif sheet.cell(row = 1, column = col).value == 'Network IBI Coefficient of Variation':
                networkIBIcoeffVar = col
                continue                
            elif sheet.cell(row = 1, column = col).value == 'Network Normalized Duration IQR':
                networkNormDurationIQR = col
                continue                
            elif sheet.cell(row = 1, column = col).value == 'Area Under Normalized Cross-Correlation':
                areaUnderNormalizedCrossCorrelation = col
                continue
            elif sheet.cell(row = 1, column = col).value == 'Area Under Cross-Correlation':
                areaUnderCrossCorrelation = col
                continue                
            elif sheet.cell(row = 1, column = col).value == 'Width at Half Height of Normalized Cross-Correlation':
                halfWidthNormalizedCrossCorrelation = col
                continue                
            elif sheet.cell(row = 1, column = col).value == 'Width at Half Height of Cross-Correlation':
                halfWidthCrossCorrelation = col
                continue
            elif sheet.cell(row = 1, column = col).value == 'Half Width at Half Height of Normalized Cross-Correlation':
                halfWidthNormalizedCrossCorrelation = col
                continue                
            elif sheet.cell(row = 1, column = col).value == 'Half Width at Half Height of Cross-Correlation':
                halfWidthCrossCorrelation = col
                continue
            elif sheet.cell(row = 1, column = col).value == 'Synchrony Index':
                syncIndex = col
                continue                

        for page in range(0, len(sheets)):   # Loop through sheets (days)
            sheet = wb.worksheets[page]
            maxCol = sheet.max_column
            sheet.cell(row = 1, column = maxCol+1).value = "NAS"
            for row in range(2,sheet.max_row+1):
                index_score = \
                (sheet.cell(row = row, column = meanFiringRate).value * 0.78723) + \
                (sheet.cell(row = row, column = ISIcoeffVar).value * 0.87546) + \
                (sheet.cell(row = row, column = numBurstingElecs).value * 0.92474) + \
                (sheet.cell(row = row, column = burstDuration).value * 0.73062) + \
                (sheet.cell(row = row, column = numSpikesPerBurst).value * 0.92763) + \
                (sheet.cell(row = row, column = meanISIwithinBurst).value * 0.02926) + \
                (sheet.cell(row = row, column = medianISIwithinBurst).value * -0.05205) + \
                (sheet.cell(row = row, column = IBI).value * -0.27591) + \
                (sheet.cell(row = row, column = burstFreq).value * 0.59294) + \
                (sheet.cell(row = row, column = normDurationIQR).value * 0.70740) + \
                (sheet.cell(row = row, column = IBIcoeffVar).value * 0.64845) + \
                (sheet.cell(row = row, column = burstPercentage).value * 0.93895) + \
                (sheet.cell(row = row, column = networkBurstFreq).value * 0.50654) + \
                (sheet.cell(row = row, column = networkBurstDuration).value * 0.49079) + \
                (sheet.cell(row = row, column = numSpikesPerNetworkBurst).value * 0.90668) + \
                (sheet.cell(row = row, column = numElecsParticipatingInBurst).value * 0.86659) + \
                (sheet.cell(row = row, column = numSpikesPerNetworkBurstPerChannel).value * 0.91682) + \
                (sheet.cell(row = row, column = networkBurstPercentage).value * 0.93034) + \
                (sheet.cell(row = row, column = networkIBIcoeffVar).value * 0.73943) + \
                (sheet.cell(row = row, column = networkNormDurationIQR).value * 0.61389) + \
                (sheet.cell(row = row, column = areaUnderNormalizedCrossCorrelation).value * 0.86675) + \
                (sheet.cell(row = row, column = areaUnderCrossCorrelation).value * 0.60665) + \
                (sheet.cell(row = row, column = halfWidthNormalizedCrossCorrelation).value * 0.17513) + \
                (sheet.cell(row = row, column = halfWidthCrossCorrelation).value * 0.23308) + \
                (sheet.cell(row = row, column = syncIndex).value * 0.91299)
                sheet.cell(row = row, column = maxCol+1).value = index_score

        # Compile NAS in GraphPad format
        # Create sheet with only Rep, Group, Day, Index Score columns
        wb.create_sheet('NAS')
        isSheet = wb['NAS']
        for i in range(1,sheet.max_row+1):
            for j in [1,2,3,4]:
                isSheet.cell(row = i, column = j).value = sheet.cell(row = i, column = j).value
            isSheet.cell(row = i, column = 5).value = sheet.cell(row = i, column = maxCol+1).value

        # Remove outliers
        indexScoreData = isSheet.values
        indexScoreDataHeaders = next(indexScoreData)[0:]
        isDF = pd.DataFrame(indexScoreData, columns=indexScoreDataHeaders) # Convert to dataframe
        isDF['NAS (z)'] = stats.zscore(isDF['NAS']) # Z-score NAS column
        isDFz = isDF['NAS (z)'].tolist()
        isSheet.cell(row = 1, column = 6).value = 'NAS (z)'
        for i in range(2, len(isDFz)+2):
            isSheet.cell(row = i, column = 6).value = isDFz[i-2]

        # Scale to 0-1
        isSheet.cell(row = 1, column = 7).value = 'NAS (scaled)'
        isVals = [isSheet.cell(row = x, column = 5).value for x in range(2,isSheet.max_row+1) if isSheet.cell(row = x, column = 6).value < 3 and isSheet.cell(row = x, column = 6).value > -3]
        minISval = min(isVals)
        maxISval = max(isVals)
        for i in range(2,isSheet.max_row+1):
            if isSheet.cell(row = i, column = 6).value > 3 or isSheet.cell(row = i, column = 6).value < -3:
                isSheet.cell(row = i, column = 7).value = 'Outlier'
            else:
                isSheet.cell(row = i, column = 7).value = (isSheet.cell(row = i, column = 5).value - minISval)/(maxISval - minISval)

        # Convert to dataFrame
        indexScoreData = isSheet.values
        indexScoreDataHeaders = next(indexScoreData)[0:]
        dfIndexScore = pd.DataFrame(indexScoreData, columns=indexScoreDataHeaders)

        wb.create_sheet('NAS scaled - GraphPad format') # Create sheet for GraphPad format
        gpSheet = wb['NAS scaled - GraphPad format']
        dfIndexScore.sort_values(by=['Group', 'Day', 'Rep'], inplace=True) # Sort index score dataframe
        repList = []
        groupList = []
        dayList = []
        for index, row in dfIndexScore.iterrows(): # Loop through rows of sorted dataframe
            if row[0] not in repList: # Create list of reps
                repList.append(row[0])
            if row[1] not in groupList: # Create list of groups
                groupList.append(row[1])
            if row[2] not in dayList: # Create list of days
                dayList.append(row[2])
        
        # Fill in GraphPad format sheet
        gpSheet.cell(row = 1, column = 1).value = 'DIV'
        for i in range(2,len(dayList)+2):
            gpSheet.cell(row = i, column = 1).value = dayList[i-2]
        groupCount, dayCount, repCount = 1, 1, 1
        for index, row in dfIndexScore.iterrows():
            gpSheet.cell(row = 1, column = ((groupCount-1)*len(repList)) + repCount + 1).value = row[1]
            if groupCount < len(groupList)+1:
                if dayCount < len(dayList):
                    if repCount < len(repList):
                        gpSheet.cell(row = dayCount+1, column = ((groupCount-1)*len(repList)) + repCount + 1).value = row[6]
                        repCount += 1
                    elif repCount == len(repList):
                        gpSheet.cell(row = dayCount+1, column = ((groupCount-1)*len(repList)) + repCount + 1).value = row[6]
                        dayCount += 1 # Add to day counter
                        repCount = 1 # Reset rep counter
                elif dayCount == len(dayList):
                    if repCount < len(repList):
                        gpSheet.cell(row = dayCount+1, column = ((groupCount-1)*len(repList)) + repCount + 1).value = row[6]
                        repCount += 1
                    elif repCount == len(repList):
                        gpSheet.cell(row = dayCount+1, column = ((groupCount-1)*len(repList)) + repCount + 1).value = row[6]
                        gpSheet.merge_cells(start_row=1, start_column=((groupCount-1)*len(repList) + 2), \
                        end_row=1, end_column=((groupCount-1)*len(repList) + 1 + len(repList))) # ...Merge group header cells before moving to next group
                        groupCount += 1 # Add to group counter
                        dayCount = 1 # Reset day counter
                        repCount = 1 # Reset rep counter
            elif groupCount == len(groupList)+1:
                break

        wb.save(platePath + '\\' + '4_NAS_' + plateFolder + '.xlsx')
    print('NAS Excel files exported.')

In [ ]:
NAS(experimentPath)
print('Analysis complete.')